In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import glob
import warnings 
warnings.filterwarnings('ignore')
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
options = webdriver.ChromeOptions()
# options.add_argument(r"C:\Users\Asif-LFD\AppData\Local\Google\Chrome\User Data\Profile 1")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# from selenium import webdriver
from selenium.webdriver.common.by import By
import pytesseract
# from PIL import Image
# import io
# import cv2
# import pytesseract
import numpy as np
import re



In [2]:
url = "https://pk.indeed.com/"
driver.get(url)
time.sleep(2)

# input keyword (input variable may be 1. job title 2. where) 
1. clear() kaam nahi kar rha dyk lyna bad me

In [3]:
var_key_word_job = "frontend"
var_key_word_where = "karachi"
var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[1]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_job)
time.sleep(2)

var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[2]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_where)
time.sleep(2)

# click find jon button
driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/button").click()


In [4]:
print("There are "+ driver.find_element("xpath","/html/body/main/div/div[1]/div/div/div[5]/div[1]/div[4]/div/div/div[2]/span[1]").text)

There are 130 jobs


# left side me companies ki details jo ati list wise os ka code

1. div (id = 'mosaic-provider-jobcards') k andar "ul" ka tag id
2. ul k andar "li" ka tag {har li me indivdual company ka card}
3. li k andar div class "job_seen_beacon" os k andar sara data






In [5]:
var_cards_company_data = driver.find_element("xpath","//div[@id='mosaic-provider-jobcards']").find_elements("xpath","ul/li")

print(len(var_cards_company_data))
# saving all data into csv
var_csv_cols = ['job_title','company_name',	'location',	'salary_range',	'job_type',	'description',	'job_link',	'indeedApply',"hiringMultipleCandidates","posted_date"]


# makinf df having columns of var_csv_cols
df = pd.DataFrame(columns=var_csv_cols)


18


In [6]:
# clicking on each card company name
var_cards_company_data[10].click()


In [7]:
# temp
soup = BeautifulSoup(var_cards_company_data[10].get_attribute('innerHTML'), 'html.parser')
print(soup.prettify())

<div class="cardOutline tapItem dd-privacy-allow result job_d40b4a5002a732a0 resultWithShelf sponTapItem desktop vjs-highlight css-kyg8or eu4oa1w0">
 <div class="slider_container css-8xisqv eu4oa1w0" dir="auto">
  <div class="slider_list css-kyg8or eu4oa1w0">
   <div class="slider_item css-kyg8or eu4oa1w0">
    <div class="job_seen_beacon">
     <table cellpadding="0" cellspacing="0" class="jobCard_mainContent big6_visualChanges" role="presentation">
      <tbody>
       <tr>
        <td class="resultContent">
         <div class="css-1m4cuuf e37uo190">
          <h2 class="jobTitle css-1h4a4n5 eu4oa1w0" tabindex="-1">
           <a aria-label="full details of Frontend Developer Intern" class="jcs-JobTitle css-jspxzf eu4oa1w0" data-hide-spinner="true" data-hiring-event="false" data-jk="d40b4a5002a732a0" data-mobtk="1h3kcpslcj6df800" href="/company/Mean3/jobs/Front-End-Developer-d40b4a5002a732a0?fccid=2221e19daf2ec2e6&amp;vjs=3" id="job_d40b4a5002a732a0" role="button" target="_blank">
 

In [8]:
try:
    # find salary
    salary_range=var_cards_company_data[10].find_element("xpath","//table[@class='jobCard_mainContent big6_visualChanges']").find_element("xpath","//tr").find_element("xpath","td").find_element("xpath","div[@class='heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly']").find_element("xpath","div[@class='metadata salary-snippet-container']").text
except:
    salary_range = "undefine"

salary_range


'Rs 25,000 - Rs 30,000 a month'

In [33]:
soup = BeautifulSoup(var_cards_company_data[10].find_element("xpath","//tr[@class = 'jobCardShelf']").get_attribute('innerHTML'), 'html.parser')
# print(soup.prettify())

html = str(soup.prettify())

regex = r'td class="shelfItem ([^"]*)">'

matches = re.findall(regex, html)
next_words = [match.strip() for match in matches]

print(next_words)


[]


In [10]:
# posted date
posted_date = var_cards_company_data[10].find_element("xpath","//table[@class = 'jobCardShelfContainer big6_visualChanges']").find_element("xpath","//tr[@class = 'underShelfFooter']").find_element("xpath","//span[@class='date']").text.split("\n")[-1]
posted_date


'Active 3 days ago'

#  right portion

In [12]:
# page ka right side
var_right_side = driver.find_element("xpath","//div[@class = 'jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent--embedded css-axjo09 eu4oa1w0']")
#ab yaha sy user interction(kisi button py click karna etc etc) kahtam hun gaya just data scrape karna han so we need to use bs4. 

# bs4
soup = BeautifulSoup(var_right_side.get_attribute('innerHTML'), 'html.parser')
print(soup.prettify())




<div class="jobsearch-JobComponent-embeddedHeader">
 <div class="jobsearch-HeaderContainer">
  <div class="jobsearch-InfoHeaderContainer">
   <div>
    <div class="jobsearch-JobInfoHeader-title-container">
     <h2 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded" dir="ltr" lang="en">
      <span>
       Frontend Developer Intern
       <span class="css-1syol37 esbq1260">
        - job post
       </span>
      </span>
     </h2>
    </div>
    <div class="jobsearch-CompanyInfoContainer">
     <div class="jobsearch-CompanyInfoWithoutHeaderImage">
      <div>
       <div class="css-1qz04qx eu4oa1w0">
        <div class="css-1h46us2 eu4oa1w0">
         <div class="css-1cjkto6 eu4oa1w0" data-company-name="true">
          <a href="https://pk.indeed.com/cmp/Mean3?campaignid=mobvjcmp&amp;from=mobviewjob&amp;tk=1h3kcq52fg80j803&amp;fromjk=d40b4a5002a732a0" target="_blank">
           Mean3
          </a>
         </div>
        </div>
        <div class="css

### left up portion (jis me job title,comapny name, location and apply karny ka button han)


In [13]:
print(soup.find('div',class_="jobsearch-InfoHeaderContainer").prettify())
# text

<div class="jobsearch-InfoHeaderContainer">
 <div>
  <div class="jobsearch-JobInfoHeader-title-container">
   <h2 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded" dir="ltr" lang="en">
    <span>
     Frontend Developer Intern
     <span class="css-1syol37 esbq1260">
      - job post
     </span>
    </span>
   </h2>
  </div>
  <div class="jobsearch-CompanyInfoContainer">
   <div class="jobsearch-CompanyInfoWithoutHeaderImage">
    <div>
     <div class="css-1qz04qx eu4oa1w0">
      <div class="css-1h46us2 eu4oa1w0">
       <div class="css-1cjkto6 eu4oa1w0" data-company-name="true">
        <a href="https://pk.indeed.com/cmp/Mean3?campaignid=mobvjcmp&amp;from=mobviewjob&amp;tk=1h3kcq52fg80j803&amp;fromjk=d40b4a5002a732a0" target="_blank">
         Mean3
        </a>
       </div>
      </div>
      <div class="css-6z8o9s eu4oa1w0">
       <div>
        Karachi
       </div>
      </div>
      <div class="css-6z8o9s eu4oa1w0">
      </div>
     </div>
 

In [14]:
# job title
job_title = soup.find('h2',class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded').text.replace(" - job post","")
job_title

'Frontend Developer Intern'

In [15]:
# comapny name
company_name = soup.find("div",class_="css-1h46us2 eu4oa1w0").text #comapny name
company_name

'Mean3'

In [16]:
# location
location = soup.find("div",class_="css-6z8o9s eu4oa1w0").text #location
location


'Karachi'

In [17]:
# nechy waly code me 2 divs aa rahy. may be contry ka hun. lakin abhi pata nahi q future me kaam aa skta
soup.find_all("div",class_="css-6z8o9s eu4oa1w0")
# .text #location


[<div class="css-6z8o9s eu4oa1w0"><div>Karachi</div></div>,
 <div class="css-6z8o9s eu4oa1w0"></div>]

### left down portion (jis me job description wagera han)


In [18]:
var_soup_job_des = soup.find("div",class_='jobsearch-embeddedBody css-1omm75o eu4oa1w0')

var_soup_job_des

<div class="jobsearch-embeddedBody css-1omm75o eu4oa1w0" tabindex="-1"><div class="jobsearch-BodyContainer"><div></div><div class="jobsearch-JobComponent-description css-kyg8or eu4oa1w0" tabindex="0"><div class="mosaic mosaic-empty-zone" id="aboveFullJobDescription"></div><div class="mosaic mosaic-empty-zone" id="aboveExtractedJobDescription"></div><div class="css-804pn3 eu4oa1w0" id="jobDetailsSection"><div class="jobsearch-JobDescriptionSection-title css-4m8ia3 eu4oa1w0"><h2 class="css-ji0ia1 e1tiznh50" id="jobDetails" tabindex="-1">Job details</h2><div class="css-7c0b8 eu4oa1w0"><span>No matching <a aria-label="job preferences (opens in a new window)" class="css-6pji5a emf9s7v0" href="https://profile.indeed.com/preferences" rel="noopener" target="_blank">job preferences<svg aria-hidden="true" class="css-r5jz5s eac13zx0" fill="currentColor" focusable="false" role="img" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.504 3a.5.5 0 00-.5.5v1a.5.5 0 00.5.5h3.085l-9.59

In [19]:
# job type
job_type = var_soup_job_des.find("div",class_="css-tvvxwd ecydgvn1").text
job_type


'Internship'

In [27]:
# jobDescriptionText
description = var_soup_job_des.find("div",id="jobDescriptionText").text
# save in text file
with open("jobDescriptionText.txt","w",encoding="utf-8") as f:
    f.write(var_soup_job_des.find("div",id="jobDescriptionText").text)
    f.close()
description

'\nMust have done one internship or have good knowledge of Frontend developer\nGraduate\nTimings: 11:00 am -6:00 pm (Ramadan: 10:00 am -4:00 pm)\n\nJob Type: InternshipContract length: 3 months\nAbility to commute/relocate:\n\nKarachi: Reliably commute or planning to relocate before starting work (Required)\n'

In [21]:
# getting link
job_link = driver.current_url
job_link

'https://pk.indeed.com/jobs?q=frontend&l=karachi&from=searchOnHP&vjk=d40b4a5002a732a0'

In [22]:

# making an empty dictionary on above listt
var_dict = {i:'' for i in var_csv_cols}
var_dict

{'job_title': '',
 'company_name': '',
 'location': '',
 'salary_range': '',
 'job_type': '',
 'description': '',
 'job_link': '',
 'indeedApply': '',
 'hiringMultipleCandidates': '',
 'posted_date': ''}

In [37]:
# var_dict[i]
next_words

[]

In [35]:
# if next_words have any of var_dict keys than put 1 of each keys values in var_dic
for i in next_words:
    if i in var_dict.keys():
        var_dict[i] = 1
    else:
        var_dict[i] = 0

var_dict

{'job_title': 'Frontend Developer Intern',
 'company_name': 'Mean3',
 'location': 'Karachi',
 'salary_range': 'Rs 25,000 - Rs 30,000 a month',
 'job_type': 'Internship',
 'description': '\nMust have done one internship or have good knowledge of Frontend developer\nGraduate\nTimings: 11:00 am -6:00 pm (Ramadan: 10:00 am -4:00 pm)\n\nJob Type: InternshipContract length: 3 months\nAbility to commute/relocate:\n\nKarachi: Reliably commute or planning to relocate before starting work (Required)\n',
 'job_link': 'https://pk.indeed.com/jobs?q=frontend&l=karachi&from=searchOnHP&vjk=d40b4a5002a732a0',
 'indeedApply': '',
 'hiringMultipleCandidates': '',
 'posted_date': 'Active 3 days ago'}

In [32]:
var_dict.keys()

dict_keys(['job_title', 'company_name', 'location', 'salary_range', 'job_type', 'description', 'job_link', 'indeedApply', 'hiringMultipleCandidates', 'posted_date'])

In [38]:
var_dict["job_title"] =job_title

var_dict["company_name"] =company_name
var_dict["location"] =location
var_dict["salary_range"] =salary_range

var_dict["job_type"] =job_type
var_dict["description"] = description
var_dict["job_link"] =job_link
# var_dict["indeedApply"] = indeedApply
# var_dict["hiringMultipleCandidates"] = hiringMultipleCandidates


var_dict["posted_date"] =posted_date


print(var_dict)


{'job_title': 'Frontend Developer Intern', 'company_name': 'Mean3', 'location': 'Karachi', 'salary_range': 'Rs 25,000 - Rs 30,000 a month', 'job_type': 'Internship', 'description': '\nMust have done one internship or have good knowledge of Frontend developer\nGraduate\nTimings: 11:00 am -6:00 pm (Ramadan: 10:00 am -4:00 pm)\n\nJob Type: InternshipContract length: 3 months\nAbility to commute/relocate:\n\nKarachi: Reliably commute or planning to relocate before starting work (Required)\n', 'job_link': 'https://pk.indeed.com/jobs?q=frontend&l=karachi&from=searchOnHP&vjk=d40b4a5002a732a0', 'indeedApply': '', 'hiringMultipleCandidates': '', 'posted_date': 'Active 3 days ago'}


In [29]:
# adding var_dict to df
df = df.append(var_dict,ignore_index=True)
df

,job_title,company_name,location,salary_range,job_type,description,job_link,indeedApply,hiringMultipleCandidates,posted_date
0,Frontend Developer Intern,Mean3,Karachi,,Internship,\nMust have done one internship or have good k...,https://pk.indeed.com/jobs?q=frontend&l=karach...,,,
1,Frontend Developer Intern,Mean3,Karachi,"Rs 25,000 - Rs 30,000 a month",Internship,\nMust have done one internship or have good k...,https://pk.indeed.com/jobs?q=frontend&l=karach...,,,Active 3 days ago


In [39]:
df.to_csv("extracted_data.csv",index=False)

In [40]:
import re

def keyword_matching(resume, job_description):
    resume_keywords = set(re.findall(r'\b\w+\b', resume.lower()))
    job_keywords = set(re.findall(r'\b\w+\b', job_description.lower()))
    matched_keywords = resume_keywords.intersection(job_keywords)
    return matched_keywords

# Example usage
resume_text = '''

EHTISHAM ALI
03412373830 | etsali232@gmail.com | linkedin.com/in/ali-ehtisham/ | github.com/ehtish
Objective
Skilled web developer with expertise in HTML, CSS, JavaScript, Bootstrap, and React. Passionate about creating visually stunning and user-friendly web experiences. Seeking a challenging role to leverage MERN stack proficiency and drive impactful solutions that exceed client expectations.
Experience
CONTOUR SOFTWARE
MERN stack trainee
- During the comprehensive MERN (MongoDB, Express, React, Node) stack development training, hands-on experience was gained in building and deploying responsive web applications.
- Worked under the supervision of industry experts to provide solutions to real-world problems.
- Developed proficiency in front-end development, creating dynamic user interfaces using React.
- Furthermore, valuable knowledge and extensive practical experience were diligently acquired in API building and proficiently handling hosting operations.
Projects
1. Exercise Tracker (Final Project at CONTOUR)
- The Exercise Tracker project utilizes the MERN stack, including MongoDB, Express, React, and Node.js, along with Bootstrap and Material UI. It offers a visually appealing, user-friendly experience with a responsive design, catering to various devices. Secure login is implemented using JWT tokens for seamless activity management.
2. Managee
- Efficiently developing the front-end of the software team management website, the project showcased proficient utilization of HTML, CSS, JavaScript, jQuery, and the feature-rich JavaScript library, Owl Carousel. With a strong emphasis on responsive design, the website ensures a seamless user experience and intuitive features.
- Live demo: https://ehtish.github.io/managee/
3. Mr. Mashwara
- Developed a chrome extension project utilizing the advice API with skillful implementation of HTML5, CSS3, JavaScript, and JSON.
- Live demo: https://github.com/Ehtish/Mr.Mashwara
4. Fylo
- Skillfully created Fylo, a file securing website, utilizing HTML5 and CSS3 with a special feature of dynamically responsive font size functionality, showcasing technical expertise and innovation.
- Live demo: https://ehtish.github.io/fylo-dark/
5. Portfolio
- Recently collaborated on a dynamic portfolio project that demonstrated proficiency in HTML5, CSS3, Bootstrap 4, and JavaScript. Impressively showcased front-end skills through custom styling, responsive design, and engaging interactive features.
- Live demo: https://ehtish.github.io/asif-portfolio/
- Note: Explore more projects and contributions on my GitHub profile: https://github.com/Ehtish
Academics
BSCS
SINDH MADRASATUL ISLAM UNVERSITY
Jan 2016 -2020
Skills
- HTML5, CSS3, Javascript, Bootstrap, React, Node, Express, and MongoDB.
Soft Skills
- Communication, Problem Solving, Adaptability, Teamwork and Time Management.
Platforms - Git, GitHub, Netlify and MySQL.
Beyond Academics - Reading Books and Gaming.
'''

job_description_text = '''
A Frontend Developer is responsible for creating and implementing the visual and interactive elements of a website or application. They work closely with designers, back-end developers, and stakeholders to ensure that the user experience is seamless and intuitive. Frontend Developers use a combination of programming languages and tools to bring design concepts to life, and must be able to balance technical skills with creativity and attention to detail.

Key Responsibilities:
- Collaborate with designers, back-end developers, and stakeholders to develop and implement user-friendly interfaces and interactive features
- Write clean, efficient, and well-documented code using HTML, CSS, and JavaScript
- Hands-on experience with JS libraries and frameworks like ReactJS, VueJS, Angular
- Should be able to connect APIs with frontend shared by backend team
- Optimize web pages for maximum speed and scalability
- Ensure cross-browser and cross-device compatibility
- Identify and troubleshoot issues related to performance, user experience, and accessibility
- Stay up-to-date with emerging trends and technologies in web development
- Conduct usability testing and gather feedback from users to improve website or application design
- Communicate technical information to non-technical stakeholders

Qualifications:
- Bachelor's degree in Computer Science, Software Engineering, or related field
- Proven experience as a Frontend Developer or similar role
- Proficiency in HTML, CSS, JavaScript, and related frameworks such as React, Angular, or Vue.js
- Experience with responsive design, mobile-first development, and cross-browser compatibility
- Knowledge of user experience (UX) design principles and methodologies
- Familiarity with version control tools such as Git
- Strong problem-solving and analytical skills
- Excellent communication and teamwork abilities

Preferred Qualifications:
- Experience with web performance optimization techniques
- Familiarity with server-side scripting languages such as PHP or Node.js
- Knowledge of search engine optimization (SEO) best practices
- Experience with web accessibility standards and guidelines (WCAG)
- Strong portfolio showcasing previous web development projects

Job Type: Full-tim
'''

matched_keywords = keyword_matching(resume_text, job_description_text)

print("Matched Keywords:", matched_keywords)
print("Number of Matches:", len(matched_keywords))


Matched Keywords: {'of', 'interactive', 'git', 'portfolio', 'development', 'showcasing', 'creating', 'proficiency', 'developer', 'to', 'end', 'communication', 'strong', 'seamless', 'with', 'react', 'skills', 'solving', 'node', 'on', 'the', 'friendly', 'html', 'using', 'problem', 'projects', 'hands', 'interfaces', 'a', 'responsive', 'technical', 'role', 'for', 'web', 'css', 'that', 'intuitive', 'website', 'knowledge', 'design', 'software', 'and', 'javascript', 'js', 'team', 'teamwork', 'in', 'features', 'is', 'experience', 'user'}
Number of Matches: 51


In [45]:
pip install rake-nltk


Note: you may need to restart the kernel to use updated packages.


In [47]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Asif-
[nltk_data]     LFD\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [48]:
import nltk
from rake_nltk import Rake

def extract_keywords(job_description):
    # Initialize Rake-NLTK
    r = Rake()

    # Extract keywords
    r.extract_keywords_from_text(job_description)

    # Get the ranked keywords
    ranked_keywords = r.get_ranked_phrases()

    return ranked_keywords

# Example usage
# job_description_text = "Job description text here"
keywords = extract_keywords(job_description_text)

# Print the extracted keywords
for keyword in keywords:
    print(keyword)


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Asif-LFD/nltk_data'
    - 'c:\\Users\\Asif-LFD\\anaconda3\\nltk_data'
    - 'c:\\Users\\Asif-LFD\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\Asif-LFD\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Asif-LFD\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
import re

def keyword_matching(resume, job_description):
    resume_keywords = set(re.findall(r'\b\w+\b', resume.lower()))
    job_keywords = set(re.findall(r'\b\w+\b', job_description.lower()))
    matched_keywords = resume_keywords.intersection(job_keywords)
    return matched_keywords

def calculate_percentage_match(resume, job_description):
    total_keywords_in_job = len(re.findall(r'\b\w+\b', job_description.lower()))
    matched_keywords_count = len(keyword_matching(resume, job_description))
    percentage_match = (matched_keywords_count / total_keywords_in_job) * 100
    return round(percentage_match, 2)

# Example usage
resume_text = "Your resume text here"
job_description_text = "Job description text here"
percentage_match = calculate_percentage_match(resume_text, job_description_text)
print("Percentage Match:", percentage_match)
percentage_match = (matched_keywords_count / total_keywords_in_job) * 100
